### Data Scrapping of earnings Call transcripts Q1 2019

#### This Notebook deals with scrapping the latest earning call transcipts of Tesla,Amazon,facebook,Goldman,Netflix,Krogger from seeking alpha.com through python library - BeautifulSoup

In [ ]:
import requests  
from bs4 import BeautifulSoup as BS
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pandas import Series, DataFrame

##### Function to get the data for each company through URL passed

In [3]:
def fetch_urldata(url):
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'referrer': 'https://google.com',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Pragma': 'no-cache',}
    r = requests.get(url,headers = headers)  
    soup = BS(r.content, 'lxml')
    
    
    arr = []
    i = 0
    for data in soup.findAll('p'):
        i = i + 1
        if (data.find('strong') != None):
            Object = {
            "text":  data.find('strong').text[0:]

    #         "Sentiment": ''        
        }
        else:
            Object = {
            "text": 
                 data.contents[0] 
    #         "Sentiment": ''
        }

        arr.append(Object)
    return arr

##### Function to determine the sentiment of each document
###### Create the datframe of documents along with their sentiments ( + , _ , neutral)

In [4]:
def dataframe_jsonfile(arr):
    df = pd.DataFrame(arr)
    analyzer = SentimentIntensityAnalyzer()
    dtext = []
    vs_pos = []
    vs_neu = []
    vs_neg = []
    
    for i in range(0, len(df)):
        dtext.append(df['text'][i])
        vs_pos.append(analyzer.polarity_scores(df['text'][i])['pos'])
        vs_neu.append(analyzer.polarity_scores(df['text'][i])['neu'])
        vs_neg.append(analyzer.polarity_scores(df['text'][i])['neg'])
    
    
    final_df = DataFrame({'text': dtext,
                    'Positive': vs_pos,
                    'Neutral': vs_neu,
                    'Negative': vs_neg})
    
    final_df = final_df[['text','Positive', 'Neutral', 'Negative']]
    
    final_df.loc[final_df.Neutral == 1, 'sentiment'] = 'Neutral'
    final_df.loc[(final_df['Positive'] > 0.094 ) & (final_df['Positive'] <= 0.9) , 'sentiment'] = 'Positive'
    final_df.loc[(final_df['Negative'] <= 0.094 ) & (final_df['Negative'] > 0) , 'sentiment'] = 'Negative'
    return final_df

##### Following Steps are performed to fetch the call transcript for each company:
###### 1)URL of seeking alpha having call transcript link is passed to the function fetch_urldata           
###### 2) The data fetched is then passed to funtion dataframe_jsonfile for sentiment analysis and  a dataframe of the documents along with their sentiments is formed                   
###### 3)Count of each sentiment is taken                     
###### 4) Dataframe is checked for any null values , if any null value is present then the documnet is analyzed manually and a sentiment is applied to it                  
###### 5) The dataframe is downloaded in the json format            
######                                              This process is repeated for all the companies from 1 - 6  below:

###### 1. Tesla

In [110]:
teslaaarr = fetch_urldata('https://seekingalpha.com/article/4256560-tesla-inc-tsla-ceo-elon-musk-q1-2019-results-earnings-call-transcript?page=2')
tesla_df = dataframe_jsonfile(teslaaarr)    
print(tesla_df.head())
print(tesla_df['sentiment'].value_counts())


                                                text  Positive  Neutral  \
0                                Tesla, Inc (NASDAQ:       0.0      1.0   
1                               Company Participants       0.0      1.0   
2  Martin Viecha - Senior Director, Investor Rela...       0.0      1.0   
3                Elon Musk - Chief Executive Officer       0.0      1.0   
4           J.B. Straubel - Chief Technology Officer       0.0      1.0   

   Negative sentiment  
0       0.0   Neutral  
1       0.0   Neutral  
2       0.0   Neutral  
3       0.0   Neutral  
4       0.0   Neutral  
Neutral     151
Positive     66
Negative     38
Name: sentiment, dtype: int64


In [111]:
tesla_df[tesla_df['sentiment'].isnull()].count()

text         20
Positive     20
Neutral      20
Negative     20
sentiment     0
dtype: int64

In [112]:
tesla_df.iloc[51].fillna('Positive')
tesla_df.iloc[148].fillna('Positive')
tesla_df = tesla_df.fillna('Neutral')

In [113]:
tesla_df[tesla_df['sentiment'].isnull()].count()

text         0
Positive     0
Neutral      0
Negative     0
sentiment    0
dtype: int64

In [114]:
tesla_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\tesla_json.json')

###### 2 . Amazon 

In [73]:
amazonaarr = fetch_urldata('https://seekingalpha.com/article/4256968-amazon-com-inc-amzn-q1-2019-results-earnings-call-transcript')
amazon_df = dataframe_jsonfile(amazonaarr)
amazon_df.drop(columns = ['Positive','Neutral','Negative'])    
print(amazon_df.head())
print(amazon_df['sentiment'].value_counts())


                                                text  Positive  Neutral  \
0                          Amazon.com, Inc. (NASDAQ:       0.0      1.0   
1                               Company Participants       0.0      1.0   
2           Shelly Kay Pfeiffer - Investor Relations       0.0      1.0   
3  Brian Olsavsky - Senior Vice President and Chi...       0.0      1.0   
4       Dave Fildes - Director of Investor Relations       0.0      1.0   

   Negative sentiment  
0       0.0   Neutral  
1       0.0   Neutral  
2       0.0   Neutral  
3       0.0   Neutral  
4       0.0   Neutral  
Neutral     70
Positive    50
Negative    22
Name: sentiment, dtype: int64


In [79]:
amazon_df[amazon_df['sentiment'].isna()].count()

text         15
Positive     15
Neutral      15
Negative     15
sentiment     0
dtype: int64

In [80]:
amazon_df = amazon_df.fillna('Neutral')

In [81]:
amazon_df[amazon_df['sentiment'].isna()].count()

text         0
Positive     0
Neutral      0
Negative     0
sentiment    0
dtype: int64

In [82]:
amazon_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\amazon_json.json')

##### 3. Facebook

In [62]:
fbaarr = fetch_urldata('https://seekingalpha.com/article/4256521-facebooks-fb-ceo-mark-zuckerberg-q1-2019-results-earnings-call-transcript')
fb_df = dataframe_jsonfile(fbaarr)
fb_df.drop(columns = ['Positive','Neutral','Negative'])    
print(fb_df.head())
print(fb_df['sentiment'].value_counts())


                                                text  Positive  Neutral  \
0                            Facebook, Inc. (NASDAQ:       0.0      1.0   
1                               Company Participants       0.0      1.0   
2  Deborah Crawford – Vice President-Investor Rel...       0.0      1.0   
3          Mark Zuckerberg – Chief Executive Officer       0.0      1.0   
4          Sheryl Sandberg – Chief Operating Officer       0.0      1.0   

   Negative sentiment  
0       0.0   Neutral  
1       0.0   Neutral  
2       0.0   Neutral  
3       0.0   Neutral  
4       0.0   Neutral  
Neutral     102
Positive     67
Negative     39
Name: sentiment, dtype: int64


In [63]:
fb_df[fb_df['sentiment'].isna()].count()

text         19
Positive     19
Neutral      19
Negative     19
sentiment     0
dtype: int64

In [64]:
fb_df[fb_df['sentiment'].isna()].head()

,text,Positive,Neutral,Negative,sentiment
30,We’re continuing to see fast adoption of Stori...,0.072,0.928,0.000,NaN
31,"Since our last call, I’ve written about some m...",0.075,0.925,0.000,NaN
38,"So over the next few years, we’re going to reb...",0.085,0.915,0.000,NaN
40,"Similarly, reducing the permanence of data may...",0.088,0.785,0.128,NaN
57,A key part of earning back trust is increasing...,0.077,0.923,0.000,NaN


In [65]:
fb_df.iloc[96].fillna('Positive')
fb_df.iloc[40].fillna('Positive')
fb_df.iloc[64].fillna('Positive')
fb_df = fb_df.fillna('Neutral')

In [67]:
# fb_df

In [68]:
fb_df[fb_df['sentiment'].isna()].count()

text         0
Positive     0
Neutral      0
Negative     0
sentiment    0
dtype: int64

In [69]:
fb_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\facebook_json.json')

###### 4.Netflix

In [70]:
netflixaarr = fetch_urldata('https://seekingalpha.com/article/4254832-netflix-inc-nflx-ceo-reed-hastings-q1-2019-results-earnings-call-transcript')
netflix_df = dataframe_jsonfile(netflixaarr)
netflix_df.drop(columns = ['Positive','Neutral','Negative'])    
print(netflix_df.head())
print(netflix_df['sentiment'].value_counts())
# netflix_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\netflix_json.json')

                     text  Positive  Neutral  Negative sentiment
0  Netflix, Inc. (NASDAQ:       0.0      1.0       0.0   Neutral
1    Company Participants       0.0      1.0       0.0   Neutral
2       Spencer Wang - IR       0.0      1.0       0.0   Neutral
3     Reed Hastings - CEO       0.0      1.0       0.0   Neutral
4   Spencer Neumann - CFO       0.0      1.0       0.0   Neutral
Neutral     75
Positive    50
Negative    19
Name: sentiment, dtype: int64


In [72]:
netflix_df[netflix_df['sentiment'].isna()]

,text,Positive,Neutral,Negative,sentiment
10,Good afternoon and welcome to Netflix Q1 2019 ...,0.090,0.910,0.0,NaN
28,"So, Greg, maybe coming back to you on the subs...",0.041,0.959,0.0,NaN
38,"And just to follow up there, Spence, just to m...",0.051,0.949,0.0,NaN
42,Maybe one more on the quarter in the letter it...,0.074,0.926,0.0,NaN
51,"So maybe sticking with you, Reed, you had a se...",0.076,0.924,0.0,NaN
55,"Greg, I'd love to take that answer and maybe g...",0.076,0.924,0.0,NaN
62,"Yeah, sure. I can start and then others can ch...",0.091,0.909,0.0,NaN
63,The specific growth in Q2 is more concentrated...,0.072,0.928,0.0,NaN
74,I would just add on to Greg too that as you kn...,0.075,0.925,0.0,NaN
100,"Okay. Spence, I wanted to jump off the M&A que...",0.078,0.922,0.0,NaN


In [74]:
netflix_df.iloc[10].fillna('Positive')
netflix_df.iloc[62].fillna('Positive')
netflix_df = netflix_df.fillna('Neutral')

In [75]:
netflix_df[netflix_df['sentiment'].isnull()].count()

text         0
Positive     0
Neutral      0
Negative     0
sentiment    0
dtype: int64

In [76]:
netflix_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\netflix_json.json')

###### 5. Krogger 

In [93]:
kroggerarr = fetch_urldata('https://seekingalpha.com/article/4271303-kroger-co-kr-ceo-rodney-mcmullen-q1-2019-results-earnings-call-transcript')
krogger_df = dataframe_jsonfile(kroggerarr)
krogger_df.drop(columns = ['Positive','Neutral','Negative'])    
print(krogger_df.head())
print(krogger_df['sentiment'].value_counts())


                                  text  Positive  Neutral  Negative sentiment
0                The Kroger Co. (NYSE:       0.0      1.0       0.0   Neutral
1                 Company Participants       0.0      1.0       0.0   Neutral
2         Rebekah Manis - Director, IR       0.0      1.0       0.0   Neutral
3  W. Rodney McMullen - Chairman & CEO       0.0      1.0       0.0   Neutral
4                Gary Millerchip - CFO       0.0      1.0       0.0   Neutral
Positive    120
Neutral      99
Negative     42
Name: sentiment, dtype: int64


In [89]:
krogger_df[krogger_df['sentiment'].isnull()].count()

text         19
Positive     19
Neutral      19
Negative     19
sentiment     0
dtype: int64

In [94]:
krogger_df.iloc[270].fillna('Positive')
krogger_df.iloc[114].fillna('Positive')
krogger_df = krogger_df.fillna('Neutral')


In [95]:
krogger_df[krogger_df['sentiment'].isnull()].count()

text         0
Positive     0
Neutral      0
Negative     0
sentiment    0
dtype: int64

In [96]:
krogger_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\krogger_json.json')

###### 6. GoldMan Sachs

In [97]:
goldmanarr = fetch_urldata('https://seekingalpha.com/article/4262707-goldman-sachs-bdc-inc-gsbd-ceo-brendan-mcgovern-q1-2019-results-earnings-call-transcript')
goldman_df = dataframe_jsonfile(goldmanarr)
goldman_df.drop(columns = ['Positive','Neutral','Negative'])    
print(goldman_df.head())
print(goldman_df['sentiment'].value_counts())


                                 text  Positive  Neutral  Negative sentiment
0      Goldman Sachs BDC, Inc. (NYSE:       0.0      1.0       0.0   Neutral
1                Company Participants       0.0      1.0       0.0   Neutral
2        Katherine Schneider - VP, IR       0.0      1.0       0.0   Neutral
3  Brendan McGovern - President & CEO       0.0      1.0       0.0   Neutral
4                     Jon Yoder - COO       0.0      1.0       0.0   Neutral
Neutral     69
Negative    37
Positive    17
Name: sentiment, dtype: int64


In [100]:
goldman_df[goldman_df['sentiment'].isnull()].count()

text         13
Positive     13
Neutral      13
Negative     13
sentiment     0
dtype: int64

In [101]:
goldman_df = goldman_df.fillna('Neutral')

In [102]:
goldman_df[goldman_df['sentiment'].isnull()].count()

text         0
Positive     0
Neutral      0
Negative     0
sentiment    0
dtype: int64

In [103]:
goldman_df.to_json(r'D:\AI Assignments\Assignment3\JSON_Files\goldman_json.json')